<a href="https://colab.research.google.com/github/Dohy-Lee/Do_it-BERT_GPT/blob/main/generation_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ratsnlp
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 968 kB/s 
     |████████████████████████████████| 2.8 MB 8.8 MB/s 
     |████████████████████████████████| 57 kB 6.8 MB/s 
     |████████████████████████████████| 582 kB 74.3 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 419 kB 72.9 MB/s 
     |████████████████████████████████| 141 kB 72.9 MB/s 
     |████████████████████████████████| 3.3 MB 28.7 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 
     |████████████████████████████████| 880 kB 61.4 MB/s 
     |████████████████████████████████| 96 kB 6.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=42091920ca5666075a82b9f35a8188039b91704148a9b8c1956ea20e57e4efe6
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Suc

##모델 환경 설정

In [10]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
from ratsnlp import nlpbook

args=GenerationTrainArguments(
    pretrained_model_name='skt/kogpt2-base-v2',
    downstream_corpus_name='nsmc',
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation',
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available else 8,
    seed=7
)
nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cp

set seed: 7


##말뭉치 다운로드

In [5]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 95.4MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 38.1MB/s]                            


##토크나이저 준비

In [11]:
from transformers import PreTrainedTokenizerFast
tokenizer=PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token='</s>', #문장 마지막에 붙이는 스폐셜 토큰(SKT가 모델을 프리트레인할 때 이렇게 지정하였으므로 똑같이 지정해줌)
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


##데이터 전처리

In [12]:
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
corpus = NsmcCorpus()
train_dataset=GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='train',
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 7.714 s]
INFO:ratsnlp:tokenize sentences [took 7.714 s]
INFO:ratsnlp:tokenize sentences [took 7.714 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:rat

In [13]:
train_dataset[0]
#input_ids : 극성 정보가 포함된 입력 문장을 토큰화한 뒤 인덱싱한 결과물, 1은 eos_token(</s>)에 대응되며, 패딩 역할
#attention_mask : 해당 토큰이 패딩 토큰(0)인지 아닌지(1) 표현
#token_type_ids : 세그먼트 정보 표현
#labels : input_ids와 같음 

GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

##학습 데이터 로더 구축

In [15]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader=DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

##평가용 데이터 로더 구축

In [20]:
from torch.utils.data import SequentialSampler
val_dataset=GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='test',
)
val_dataloader=DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 1.542 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 1.542 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 1.542 s]


##모델 초기화

In [21]:
from transformers import GPT2LMHeadModel
model=GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name,
)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

##태스크 정의

In [22]:
from ratsnlp.nlpbook.generation import GenerationTask
task=GenerationTask(model,args)

##트레이너 정의 및 학습 시작

In [23]:
trainer=nlpbook.get_trainer(args)
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/optimizer.py:381: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  f"Found unsupported keys in the optimizer configuration: {set(extra_keys)}", category=RuntimeWarning
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]